In [1]:
import requests
import json
from bs4 import BeautifulSoup
import re
from datetime import datetime,timedelta
import time
import pandas as pd
from dateutil.relativedelta import relativedelta
from datetime import date

next_page가 1이면 다음페이지가 있다는 의미에요.

next_max_id는 다음페이지를 열기 위한 id입니다.

 

sections에는 게시물 데이터들이 들어있어요.

sections의 타입은 리스트입니다.

이런식으로 처음에는 어느 키에 어떤 값들이 들어가있는지 확인해보는 것이 좋아요.

In [2]:
#검색할 키워드 작성
searh_words = '제주도여행'

In [3]:
url = 'https://i.instagram.com/api/v1/tags/web_info/'
params = {
    'tag_name':searh_words
         }
headers = {
    'authority': 'i.instagram.com',
    'method': 'GET',
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.9',
    'cookie': 'mid=YtT-CgALAAEKc_oIe-ryepRR93aM; ig_did=13F8E571-D738-44F2-926C-3C4FC59E7AF8; ig_nrcb=1; csrftoken=Y0uQnYM3RGEl3ysCe5bHshk03z2YoyEr; ds_user_id=54127204025; sessionid=54127204025%3A2VFyzNnlttnXDp%3A22%3AAYfVu6nm5lsQ5lolF5Kr3x6FnLK2e8YzFPcf1SUUaQ; dpr=3.5; rur="PRN\05454127204025\0541689666138:01f789c9e1dcec8cf88dd67c24bd7f6f7d90486c12d7c90ba1604f687bb3b6474961f976\"',
    'origin': 'https://www.instagram.com',
    'referer': 'https://www.instagram.com/',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Linux; Android 10; SM-G981B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Mobile Safari/537.36',
    'x-asbd-id': '198387',
    'x-csrftoken': 'Y0uQnYM3RGEl3ysCe5bHshk03z2YoyEr',
    'x-ig-app-id': '1217981644879628',
    'x-ig-www-claim': 'hmac.AR31KBD6u5N7Ri3WWfyM2182qS6Kwk3q3fAgWbwJc'
}

In [4]:
def get_json_from_instagram(url,params,headers):
    response = requests.get(url,params=params,headers=headers)
    if response.status_code == 200:
        text = response.json()
    else:
        print('인스타그램에서 json파일을 가져오지 못했습니다.')
    return text

In [22]:
def get_results_from_json(json_text):
    sections = json_text['data']['recent']['sections']
    next_max_id = json_text['data']['recent']['next_max_id']
    next_media_ids = json_text['data']['recent']['next_media_ids']
    next_page = json_text['data']['recent']['next_page']
    result = []
    for section in sections:
        medias = section['layout_content']['medias']
        for media_list in medias:
            tmp_dict = dict()
            media = media_list['media']
            #code들 (instagram 게시물 경로)
            code = media.get('code')
            tmp_dict['code'] = code
            #장소
            location=media.get('location')
            tmp_dict['location'] = location
            #시간 datetime
            take_at=media.get('taken_at')
            readable = time.ctime(take_at)
            taken_at=datetime.strptime(readable,'%a %b %d %H:%M:%S %Y')
            tmp_dict['taken_at']=taken_at
            #tag들 
            caption= media.get('caption')
            if caption is not None:
                tag_text = caption.get('text')
                tag = tag = re.findall('#[A-Za-z0-9가-힣]+',tag_text)
                tmp_dict['tag']=tag
            else:
                tmp_dict['tag'] = None
            #사진에대한 instagram ai?가 달아놓은 간단한 설명
            accessibility_caption = media.get('accessibility_caption')
            tmp_dict['accessibility_caption'] = accessibility_caption
            #댓글 숫자
            comment_count = media.get('comment_count')
            tmp_dict['comment_count'] = comment_count
            #좋아요 숫자
            like_count = media.get('like_count')
            tmp_dict['like_count'] = like_count
            #결과 창에 추가
            result.append(tmp_dict)    
            
    return result,next_max_id,next_media_ids,next_page

In [6]:
def condition_of_crawling(json_text):
    sections = json_text['data']['recent']['sections']
    next_page = json_text['data']['recent']['next_page']
    next_max_id = json_text['data']['recent']['next_max_id']
    
    #오늘 날짜 기준으로 1주일전  1주일이내의 게시물만 크롤링 해올 예정
    week_before= datetime.combine(date.today() - relativedelta(weeks=1), datetime.min.time())
    
    last_taken_at=sections[-1]['layout_content']['medias'][-1]['media'].get('taken_at')
    last_readable = time.ctime(last_taken_at)
    last_taken_at=datetime.strptime(last_readable,'%a %b %d %H:%M:%S %Y')
    condition_result = last_taken_at >= week_before
    
    return condition_result

In [7]:
page_num = 1
j_text=get_json_from_instagram(url=url,params=params,headers=headers)
result = get_results_from_json(j_text)
real_result = result[0]
print(page_num,' 페이지 완료')
time.sleep(2)
while condition_of_crawling(j_text):
    params['max_id'] = result[1]
    j_text = get_json_from_instagram(url=url,params=params,headers=headers)
    result = get_results_from_json(j_text)
    real_result = real_result + result[0]
    page_num += 1
    print(page_num,' 페이지 완료')
    time.sleep(2)

1  페이지 완료
2  페이지 완료
3  페이지 완료
4  페이지 완료
5  페이지 완료
6  페이지 완료
7  페이지 완료
8  페이지 완료


In [8]:
help(requests.Session)

Help on class Session in module requests.sessions:

class Session(SessionRedirectMixin)
 |  A Requests session.
 |  
 |  Provides cookie persistence, connection-pooling, and configuration.
 |  
 |  Basic Usage::
 |  
 |    >>> import requests
 |    >>> s = requests.Session()
 |    >>> s.get('https://httpbin.org/get')
 |    <Response [200]>
 |  
 |  Or as a context manager::
 |  
 |    >>> with requests.Session() as s:
 |    ...     s.get('https://httpbin.org/get')
 |    <Response [200]>
 |  
 |  Method resolution order:
 |      Session
 |      SessionRedirectMixin
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __getstate__(self)
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __setstate__(self, state)
 |  
 |  close(self)
 |      Closes all adapters and as such the session
 |  
 |  delete(self, url, **kwargs)
 |      Sends a DELETE request. Returns :class:`Re

In [9]:
s = requests.Session()

In [31]:
resp=requests.get(url=url,params=params,headers=headers)

In [11]:
resp.status_code

200

In [32]:
text_1 = resp.json()
result1= get_results_from_json(text_1)

In [33]:
max_id1=result1[1]
next_media_ids1=result1[2]
page = result1[3]

In [76]:
max_id1.split('==')[0]+'%3D%3D'

'QVFETmltXy1ldm0wU19MWVk0MU92bHBVZnF0SDhzQmFybVhyVUpCRV9GblVlREJyVHRVcE5NTzVCRFlHcHQ4SUc4dUk5X0FhTFhnbXRmWXQyQjVtN1lNTQ%3D%3D'

In [35]:
next_media_ids1

['2885510820679999299', '2885461915490718786']

In [36]:
page

1

In [46]:
resp.url.split('?tag_name=')[-1]

'%EC%A0%9C%EC%A3%BC%EB%8F%84%EC%97%AC%ED%96%89'

In [51]:
url2 = 'https://i.instagram.com/api/v1/tags/'+resp.url.split('?tag_name=')[-1]+'/sections/'

In [52]:
url2

'https://i.instagram.com/api/v1/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EC%97%AC%ED%96%89/sections/'

In [92]:
data={
    'include_persistent': 0,
    'max_id': max_id1,
    'next_media_ids[]': next_media_ids1,
    'page': page,
    'surface': 'grid',
    'tab': 'recent'
}

In [73]:
headers2 = headers

In [93]:
headers2['method'] = 'POST'
headers2['x-instagram-ajax']= '1005868904'
headers2['content-length'] = '265'
headers2['content-type']= 'application/x-www-form-urlencoded'


In [94]:
resp2=s.post(url=url2,data=data,headers=headers2,)
resp2.status_code

200

In [95]:
resp2.json()

{'sections': [{'layout_type': 'media_grid',
   'layout_content': {'medias': [{'media': {'taken_at': 1658210759,
       'pk': '2885603451783296019',
       'id': '2885603451783296019_2204673139',
       'device_timestamp': 31197471364531,
       'media_type': 1,
       'code': 'CgLuQ0CLzQT',
       'client_cache_key': 'Mjg4NTYwMzQ1MTc4MzI5NjAxOQ==.2',
       'filter_type': 0,
       'accessibility_caption': 'Photo by 우도올레보트 ~ 우도다이브 ~ 고양이선장까페 in 우도올레보트 udo olleh boat. May be an image of 4 people and people standing.',
       'is_unified_video': False,
       'location': {'pk': 104676768101786,
        'short_name': '우도올레보트 udo olleh boat',
        'facebook_places_id': 104676768101786,
        'external_source': 'facebook_places',
        'name': '우도올레보트 udo olleh boat',
        'address': '제주 제주시 우도면 우도해안길 181',
        'city': '',
        'has_viewer_saved': False,
        'lng': 126.94435592259,
        'lat': 33.495995959915,
        'is_eligible_for_guides': True},
       'lat': 33.

In [70]:
headers2['content-type']= 'application/x-www-form-urlencoded'


In [56]:
result = []
for section in sections:
    medias = section['layout_content']['medias']
    for media_list in medias:
        tmp_dict = dict()
        media = media_list['media']
        #code들 (instagram 게시물 경로)
        code = media.get('code')
        tmp_dict['code'] = code
        #장소
        location=media.get('location')
        tmp_dict['location'] = location
        #시간 datetime
        take_at=media.get('taken_at')
        readable = time.ctime(take_at)
        taken_at=datetime.strptime(readable,'%a %b %d %H:%M:%S %Y')
        tmp_dict['taken_at']=taken_at
        #tag들 
        tag_text= media['caption']['text']
        tag = tag = re.findall('#[A-Za-z0-9가-힣]+',tag_text)
        tmp_dict['tag']=tag
        #사진에대한 instagram ai?가 달아놓은 간단한 설명
        accessibility_caption = media.get('accessibility_caption')
        tmp_dict['accessibility_caption'] = accessibility_caption
        #댓글 숫자
        comment_count = media.get('comment_count')
        tmp_dict['comment_count'] = comment_count
        #좋아요 숫자
        like_count = media.get('like_count')
        tmp_dict['like_count'] = like_count
        #결과 창에 추가
        result.append(tmp_dict)
        

In [60]:
last_taken_at=sections[-1]['layout_content']['medias'][-1]['media'].get('taken_at')
last_readable = time.ctime(last_taken_at)
last_taken_at=datetime.strptime(readable,'%a %b %d %H:%M:%S %Y')

In [94]:
last_taken_at < week_before

False


In [78]:
date.today() - relativedelta(weeks=1)

datetime.date(2022, 7, 12)

In [81]:
week_before= datetime.combine(date.today() - relativedelta(weeks=1), datetime.min.time())

In [8]:
len(sections)

10

In [20]:
len(sections2)

10

In [9]:
params['next_max_id'] = next_max_id

In [10]:
response = requests.get(url,params=params,headers=headers)
if response.status_code == 200:
    text = response.json()
#최신순으로 게시물 가져옴
sections2 = text['data']['recent']['sections']
next_page2 = text['data']['recent']['next_page']
next_max_id2 = text['data']['recent']['next_max_id']

In [12]:
medias=sections[0]['layout_content']['medias']
medias2 = sections2[0]['layout_content']['medias']

In [18]:
len(medias)

3

In [19]:
len(medias2)

3

In [52]:
media= medias[0]['media']

In [53]:
media.get('accessibility_caption')

'Photo by 김쏘 in 서귀포. May be an image of 2 people, tree, road and street.'

In [38]:
#장소는 명시한 경우도 있고 아닌경우도 있음 그러므로 .get을 사용
location=media.get('location')
print(location)

None


In [35]:
#시간 datetime
take_at=media.get('taken_at')
readable = time.ctime(take_at)
taken_at=datetime.strptime(readable,'%a %b %d %H:%M:%S %Y')
taken_at

datetime.datetime(2022, 7, 19, 9, 45, 23)

In [40]:
#tag들 
tag_text= media['caption']['text']
tag = tag = re.findall('#[A-Za-z0-9가-힣]+',tag_text)
tag

['#뽀로로앤타요테마파크제주점',
 '#뽀로로테마파크',
 '#뽀로로파크제주',
 '#제주테마파크',
 '#제주맘일상',
 '#제주맘카페',
 '#제주키즈카페',
 '#제주여행',
 '#제주도민',
 '#제주도여행',
 '#제주도비올때가볼만한곳',
 '#제주실내관광지',
 '#제주아이랑가기좋은곳',
 '#아이들과가볼만한곳',
 '#제주아이들과가볼만한곳',
 '#아이와제주여행',
 '#아이와제주']

In [41]:
#code들 (instagram 게시물 경로)
code = media.get('code')
code

'CgLJkqav5IY'

<table class="docutils align-default">
<colgroup>
<col style="width: 15%">
<col style="width: 43%">
<col style="width: 32%">
<col style="width: 9%">
</colgroup>
<thead>
<tr class="row-odd"><th class="head"><p>지시자</p></th>
<th class="head"><p>의미</p></th>
<th class="head"><p>예</p></th>
<th class="head"><p>노트</p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p><code class="docutils literal notranslate"><span class="pre">%a</span></code></p></td>
<td><p>요일을 로케일의 축약된 이름으로.</p></td>
<td><div class="line-block">
<div class="line">Sun, Mon, …, Sat
(en_US);</div>
<div class="line">So, Mo, …, Sa
(de_DE)</div>
</div>
</td>
<td><p>(1)</p></td>
</tr>
<tr class="row-odd"><td><p><code class="docutils literal notranslate"><span class="pre">%A</span></code></p></td>
<td><p>요일을 로케일의 전체 이름으로.</p></td>
<td><div class="line-block">
<div class="line">Sunday, Monday, …,
Saturday (en_US);</div>
<div class="line">Sonntag, Montag, …,
Samstag (de_DE)</div>
</div>
</td>
<td><p>(1)</p></td>
</tr>
<tr class="row-even"><td><p><code class="docutils literal notranslate"><span class="pre">%w</span></code></p></td>
<td><p>요일을 10진수로, 0은 일요일이고 6은 토요일입니다.</p></td>
<td><p>0, 1, …, 6</p></td>
<td></td>
</tr>
<tr class="row-odd"><td><p><code class="docutils literal notranslate"><span class="pre">%d</span></code></p></td>
<td><p>월중 일(day of the month)을 0으로 채워진 10진수로.</p></td>
<td><p>01, 02, …, 31</p></td>
<td><p>(9)</p></td>
</tr>
<tr class="row-even"><td><p><code class="docutils literal notranslate"><span class="pre">%b</span></code></p></td>
<td><p>월을 로케일의 축약된 이름으로.</p></td>
<td><div class="line-block">
<div class="line">Jan, Feb, …, Dec
(en_US);</div>
<div class="line">Jan, Feb, …, Dez
(de_DE)</div>
</div>
</td>
<td><p>(1)</p></td>
</tr>
<tr class="row-odd"><td><p><code class="docutils literal notranslate"><span class="pre">%B</span></code></p></td>
<td><p>월을 로케일의 전체 이름으로.</p></td>
<td><div class="line-block">
<div class="line">January, February,
…, December (en_US);</div>
<div class="line">Januar, Februar, …,
Dezember (de_DE)</div>
</div>
</td>
<td><p>(1)</p></td>
</tr>
<tr class="row-even"><td><p><code class="docutils literal notranslate"><span class="pre">%m</span></code></p></td>
<td><p>월을 0으로 채워진 10진수로.</p></td>
<td><p>01, 02, …, 12</p></td>
<td><p>(9)</p></td>
</tr>
<tr class="row-odd"><td><p><code class="docutils literal notranslate"><span class="pre">%y</span></code></p></td>
<td><p>세기가 없는 해(year)를 0으로 채워진 10진수로.</p></td>
<td><p>00, 01, …, 99</p></td>
<td><p>(9)</p></td>
</tr>
<tr class="row-even"><td><p><code class="docutils literal notranslate"><span class="pre">%Y</span></code></p></td>
<td><p>세기가 있는 해(year)를 10진수로.</p></td>
<td><p>0001, 0002, …, 2013,
2014, …, 9998, 9999</p></td>
<td><p>(2)</p></td>
</tr>
<tr class="row-odd"><td><p><code class="docutils literal notranslate"><span class="pre">%H</span></code></p></td>
<td><p>시(24시간제)를 0으로 채워진 십진수로.</p></td>
<td><p>00, 01, …, 23</p></td>
<td><p>(9)</p></td>
</tr>
<tr class="row-even"><td><p><code class="docutils literal notranslate"><span class="pre">%I</span></code></p></td>
<td><p>시(12시간제)를 0으로 채워진 십진수로.</p></td>
<td><p>01, 02, …, 12</p></td>
<td><p>(9)</p></td>
</tr>
<tr class="row-odd"><td><p><code class="docutils literal notranslate"><span class="pre">%p</span></code></p></td>
<td><p>로케일의 오전이나 오후에 해당하는 것.</p></td>
<td><div class="line-block">
<div class="line">AM, PM (en_US);</div>
<div class="line">am, pm (de_DE)</div>
</div>
</td>
<td><p>(1),
(3)</p></td>
</tr>
<tr class="row-even"><td><p><code class="docutils literal notranslate"><span class="pre">%M</span></code></p></td>
<td><p>분을 0으로 채워진 십진수로.</p></td>
<td><p>00, 01, …, 59</p></td>
<td><p>(9)</p></td>
</tr>
<tr class="row-odd"><td><p><code class="docutils literal notranslate"><span class="pre">%S</span></code></p></td>
<td><p>초를 0으로 채워진 10진수로.</p></td>
<td><p>00, 01, …, 59</p></td>
<td><p>(4),
(9)</p></td>
</tr>
<tr class="row-even"><td><p><code class="docutils literal notranslate"><span class="pre">%f</span></code></p></td>
<td><p>Microsecond as a decimal
number, zero-padded to 6
digits.</p></td>
<td><p>000000, 000001, …,
999999</p></td>
<td><p>(5)</p></td>
</tr>
<tr class="row-odd"><td><p><code class="docutils literal notranslate"><span class="pre">%z</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">±HHMM[SS[.ffffff]]</span></code> 형태의 UTC 오프셋 (객체가 나이브하면 빈 문자열).</p></td>
<td><p>(비어 있음), +0000, -0400, +1030, +063415, -030712.345216</p></td>
<td><p>(6)</p></td>
</tr>
<tr class="row-even"><td><p><code class="docutils literal notranslate"><span class="pre">%Z</span></code></p></td>
<td><p>시간대 이름 (객체가 나이브하면 빈 문자열).</p></td>
<td><p>(비어 있음), UTC, GMT</p></td>
<td><p>(6)</p></td>
</tr>
<tr class="row-odd"><td><p><code class="docutils literal notranslate"><span class="pre">%j</span></code></p></td>
<td><p>연중 일(day of the year)을 0으로 채워진 십진수로.</p></td>
<td><p>001, 002, …, 366</p></td>
<td><p>(9)</p></td>
</tr>
<tr class="row-even"><td><p><code class="docutils literal notranslate"><span class="pre">%U</span></code></p></td>
<td><p>Week number of the year
(Sunday as the first day of
the week) as a zero-padded
decimal number. All days in a
new year preceding the first
Sunday are considered to be in
week 0.</p></td>
<td><p>00, 01, …, 53</p></td>
<td><p>(7),
(9)</p></td>
</tr>
<tr class="row-odd"><td><p><code class="docutils literal notranslate"><span class="pre">%W</span></code></p></td>
<td><p>Week number of the year
(Monday as the first day of
the week) as a zero-padded
decimal number. All days in a
new year preceding the first
Monday are considered to be in
week 0.</p></td>
<td><p>00, 01, …, 53</p></td>
<td><p>(7),
(9)</p></td>
</tr>
<tr class="row-even"><td><p><code class="docutils literal notranslate"><span class="pre">%c</span></code></p></td>
<td><p>로케일의 적절한 날짜와 시간 표현.</p></td>
<td><div class="line-block">
<div class="line">Tue Aug 16 21:30:00
1988 (en_US);</div>
<div class="line">Di 16 Aug 21:30:00
1988 (de_DE)</div>
</div>
</td>
<td><p>(1)</p></td>
</tr>
<tr class="row-odd"><td><p><code class="docutils literal notranslate"><span class="pre">%x</span></code></p></td>
<td><p>로케일의 적절한 날짜 표현.</p></td>
<td><div class="line-block">
<div class="line">08/16/88 (None);</div>
<div class="line">08/16/1988 (en_US);</div>
<div class="line">16.08.1988 (de_DE)</div>
</div>
</td>
<td><p>(1)</p></td>
</tr>
<tr class="row-even"><td><p><code class="docutils literal notranslate"><span class="pre">%X</span></code></p></td>
<td><p>로케일의 적절한 시간 표현.</p></td>
<td><div class="line-block">
<div class="line">21:30:00 (en_US);</div>
<div class="line">21:30:00 (de_DE)</div>
</div>
</td>
<td><p>(1)</p></td>
</tr>
<tr class="row-odd"><td><p><code class="docutils literal notranslate"><span class="pre">%%</span></code></p></td>
<td><p>리터럴 <code class="docutils literal notranslate"><span class="pre">'%'</span></code> 문자.</p></td>
<td><p>%</p></td>
<td></td>
</tr>
</tbody>
</table>